<a href="https://colab.research.google.com/github/Murilo1514/Diabets_prediction_pipeline/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install optuna
!pip install ucimlrepo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00


In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import optuna
from plotly.io import show
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, FunctionTransformer, RobustScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, f1_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_score
from sklearn.compose import ColumnTransformer
from ucimlrepo import fetch_ucirepo
from functools import partial
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from imblearn.over_sampling import *
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer


# Importando a Base de dados e separando 5%



In [45]:
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

data = cdc_diabetes_health_indicators.data.features
aux = cdc_diabetes_health_indicators.data.targets

data["Diabetes_binary"] = aux["Diabetes_binary"]

#print(cdc_diabetes_health_indicators.metadata)

#print(cdc_diabetes_health_indicators.variables)

data.columns

<ipython-input-45-0eb90a80a6c7>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income',
       'Diabetes_binary'],
      dtype='object')

In [24]:
"""
data['Age']=data['Age'].astype('int64')
data['Education']=data['Education'].astype('int64')
data['Income']=data['Income'].astype('int64')
data['BMI']=data['BMI'].astype('int64')
data['CholCheck']=data['CholCheck'].astype('int64')
data['Smoker']=data['Smoker'].astype('int64')
data['Stroke']=data['Stroke'].astype('int64')
data['HeartDiseaseorAttack']=data['HeartDiseaseorAttack'].astype('int64')
data['PhysActivity']=data['PhysActivity'].astype('int64')
data['Fruits']=data['Fruits'].astype('int64')
data['Veggies']=data['Veggies'].astype('int64')
data['HvyAlcoholConsump']=data['HvyAlcoholConsump'].astype('int64')
data['AnyHealthcare']=data['AnyHealthcare'].astype('int64')
data['NoDocbcCost']=data['NoDocbcCost'].astype('int64')
data['GenHlth']=data['GenHlth'].astype('int64')
data['MentHlth']=data['MentHlth'].astype('int64')
data['PhysHlth']=data['PhysHlth'].astype('int64')
data['DiffWalk']=data['DiffWalk'].astype('int64')
data['Sex']=data['Sex'].astype('int64')
data['Diabetes_binary']=data['Diabetes_binary'].astype('int64')
data['HighBP']=data['HighBP'].astype('int64')
data['HighChol']=data['HighChol'].astype('int64')

<ipython-input-24-bc92d329addf>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-24-bc92d329addf>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-24-bc92d329addf>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-24-bc92d32

In [46]:
data.duplicated().sum()

data.drop_duplicates(inplace=True)

<ipython-input-46-1ecf93f2f3f5>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']

categorical_columns = ['GenHlth', 'Age', 'Education', 'Income']
binary_columns = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']

In [48]:
data1 = data.sample(frac=0.1, random_state=42)

x = data1.drop('Diabetes_binary', axis = 1)
y = data1['Diabetes_binary']

In [49]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.15, random_state=42, stratify=y_train)


In [50]:
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

In [51]:
preprocessor_svm = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(sparse_output= False, feature_name_combiner = 'concat'), categorical_columns),
        ('bin', 'passthrough', binary_columns),
    ])


pipeline = Pipeline(steps=[
      ('preprocessor_svm', preprocessor_svm),
      ('model', SVC())
      #('model', model)
      #('outlier_remover', outlier_remover),
      #('svm' , SVC(kernel='sigmoid', gamma = 0.01, C=733)),
  ])

###                                Oversampling

In [52]:
over_methods = [
    ADASYN(random_state=7),
    BorderlineSMOTE(random_state=7),
    RandomOverSampler(random_state=7),
    SMOTE(random_state=7),
    SVMSMOTE(random_state=7)
]

In [53]:
classifiers =[
    SVC()
]

Metricas para comparar os métodos

In [54]:
metrics = {
    'roc_auc': 'roc_auc',
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}


In [ ]:
steps = [('model', SVC(kernel='sigmoid', C=733, gamma= 0.01  ))]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=7)

for metric_name, metric in metrics.items():
    scores = cross_val_score(
        pipeline, x_train, y_train, scoring=metric, cv=cv, n_jobs=-1
    )
    print(f"{metric_name.upper()} for SVM is {round(np.mean(scores), 5)}")


ROC_AUC for SVM is 0.65854
ACCURACY for SVM is 0.78047
PRECISION for SVM is 0.29426
RECALL for SVM is 0.28721
F1 for SVM is 0.29056


In [ ]:
for classifier in classifiers:
    for method in over_methods:
        X_resampled, y_resampled = method.fit_resample(x_train, y_train)

        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', classifier)
        ])

        cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=7)

        for metric_name, metric in metrics.items():
            scores = cross_val_score(
                pipeline, X_resampled, y_resampled, scoring=metric, cv=cv, n_jobs=-1
            )
            print(f"{metric_name.upper()} for {classifier.__class__.__name__} and {method.__class__.__name__} is {round(np.mean(scores), 5)}")
        print("================================================================")


ROC_AUC for SVC and ADASYN is 0.93804
ACCURACY for SVC and ADASYN is 0.85946
PRECISION for SVC and ADASYN is 0.84634
RECALL for SVC and ADASYN is 0.87647
F1 for SVC and ADASYN is 0.86113
ROC_AUC for SVC and BorderlineSMOTE is 0.9436
ACCURACY for SVC and BorderlineSMOTE is 0.87106
PRECISION for SVC and BorderlineSMOTE is 0.85496
RECALL for SVC and BorderlineSMOTE is 0.89377
F1 for SVC and BorderlineSMOTE is 0.87392
ROC_AUC for SVC and RandomOverSampler is 0.883
ACCURACY for SVC and RandomOverSampler is 0.80567
PRECISION for SVC and RandomOverSampler is 0.77212
RECALL for SVC and RandomOverSampler is 0.86735
F1 for SVC and RandomOverSampler is 0.81696
ROC_AUC for SVC and SMOTE is 0.94098
ACCURACY for SVC and SMOTE is 0.86573
PRECISION for SVC and SMOTE is 0.85844
RECALL for SVC and SMOTE is 0.87591
F1 for SVC and SMOTE is 0.86707
ROC_AUC for SVC and SVMSMOTE is 0.92664
ACCURACY for SVC and SVMSMOTE is 0.85553
PRECISION for SVC and SVMSMOTE is 0.83235
RECALL for SVC and SVMSMOTE is 0.8904

## TESTANDO ACURÁCIA ANTES DE APLICAR O BORDERLINESMOTE

In [101]:
model = SVC()
pipeline = make_pipeline( model)

pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)

score = classification_report(y_test, y_pred)

print(f'{score}')

              precision    recall  f1-score   support

           0       0.84      1.00      0.92      3872
           1       0.50      0.00      0.01       718

    accuracy                           0.84      4590
   macro avg       0.67      0.50      0.46      4590
weighted avg       0.79      0.84      0.77      4590



In [18]:
x_train.shape

(15603, 49)

In [55]:
method = BorderlineSMOTE(random_state=7)
x_train_resampled, y_train_resampled = method.fit_resample(x_train, y_train)

x_train, y_train = x_train_resampled, y_train_resampled

In [56]:
x_train = preprocessor_svm.fit_transform(x_train)
#x_train = x_train2
x_validation = preprocessor_svm.transform(x_validation)
#x_validation = x_validation2
x_test = preprocessor_svm.transform(x_test)
#x_test = x_test2

# Para visualizar as variáveis transformadas (se precisar)
features = preprocessor_svm.get_feature_names_out()
x_train = pd.DataFrame(x_train)
print(x_train.head())
print(len(features))

         0         1         2    3    4    5    6    7    8    9   ...   39  \
0 -0.398232 -0.463225 -0.586629  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0   
1 -0.398232 -0.340325 -0.586629  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2 -0.679169 -0.463225 -0.586629  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
3 -0.117295 -0.463225 -0.586629  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
4 -1.100574 -0.463225  2.353797  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   

    40   41   42   43   44   45   46   47   48  
0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  
1  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  
2  0.0  1.0  1.0  1.0  0.0  1.0  0.0  0.0  1.0  
3  0.0  1.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  
4  0.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0  

[5 rows x 49 columns]
49


## TESTANDO ACURÁCIA DEPOIS DE APLICAR O BORDERLINESMOTE

In [36]:
model = SVC()
pipeline = make_pipeline( model)

pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)

score = classification_report(y_test, y_pred)

print(f'{score}')



              precision    recall  f1-score   support

           0       0.91      0.75      0.82      3872
           1       0.30      0.58      0.40       718

    accuracy                           0.73      4590
   macro avg       0.61      0.67      0.61      4590
weighted avg       0.81      0.73      0.76      4590



In [ ]:
model = SVC(kernel='sigmoid', C=733, gamma= 0.01  )
pipeline = make_pipeline( model)

pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)

score = classification_report(y_test, y_pred)

print(f'{score}')

              precision    recall  f1-score   support

           0       0.91      0.67      0.77      3872
           1       0.27      0.66      0.38       718

    accuracy                           0.67      4590
   macro avg       0.59      0.67      0.58      4590
weighted avg       0.81      0.67      0.71      4590



In [20]:
y_train.value_counts()

,count
Diabetes_binary,
0,13160
1,13160


In [21]:
x_train.shape

(26320, 49)

In [22]:
y_train.value_counts()

,count
Diabetes_binary,
0,13160
1,13160


## Testando acurácia antes de aplicar o PCA

In [57]:
column_names = features

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_validation = pd.DataFrame(x_validation)

In [58]:
# Contar NaNs em cada coluna
nan_counts = x_train.isnull().sum()

# Mostrar apenas as colunas com NaNs
print(nan_counts[nan_counts > 0])


Series([], dtype: int64)


In [42]:
model = SVC()
pipeline = make_pipeline(model)

pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)
score = classification_report(y_test, y_pred)

   # accuracy = accuracy_score(y_test, y_pred)

print(f'Acurácia com PCA ({i} componentes):\n {score}')

Acurácia com PCA (49 componentes):
               precision    recall  f1-score   support

           0       0.91      0.75      0.82      3872
           1       0.30      0.58      0.40       718

    accuracy                           0.73      4590
   macro avg       0.61      0.67      0.61      4590
weighted avg       0.81      0.73      0.76      4590



In [40]:
for i in range(1, 50):
    print("==========================================================")

    pca = PCA(n_components=i)

    pipeline = make_pipeline(pca, model)

    pipeline.fit(x_train, y_train)

    y_pred = pipeline.predict(x_test)

    score = classification_report(y_test, y_pred)

   # accuracy = accuracy_score(y_test, y_pred)

    print(f'Acurácia com PCA ({i} componentes): {score}')

Acurácia com PCA (1 componentes):               precision    recall  f1-score   support

           0       0.91      0.63      0.75      3872
           1       0.26      0.68      0.37       718

    accuracy                           0.64      4590
   macro avg       0.59      0.66      0.56      4590
weighted avg       0.81      0.64      0.69      4590

Acurácia com PCA (2 componentes):               precision    recall  f1-score   support

           0       0.92      0.65      0.76      3872
           1       0.27      0.69      0.38       718

    accuracy                           0.65      4590
   macro avg       0.59      0.67      0.57      4590
weighted avg       0.82      0.65      0.70      4590

Acurácia com PCA (3 componentes):               precision    recall  f1-score   support

           0       0.93      0.65      0.77      3872
           1       0.29      0.75      0.42       718

    accuracy                           0.67      4590
   macro avg       0.61   

In [63]:
def objective(trial):
    pipeline = Pipeline([
        ('preprocessor', preprocessor_svm),
        ('svm', SVC())
    ])

    C = trial.suggest_float('C', 1.0, 1000.0, log=True)  # C em escala logarítmica
    gamma = trial.suggest_float('gamma', 0.01, 2.0, step=0.005)  # Gamma
    kernel = trial.suggest_categorical('kernel', ['rbf', 'poly', 'sigmoid'])  # Kernel

    # Pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor_svm),
        ('svm', SVC())
    ])
    pipeline.set_params(svm__C=C, svm__gamma=gamma, svm__kernel=kernel)

    # Treinamento e predição
    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_validation)

    # Métrica de avaliação
    score = f1_score(y_validation, y_pred)
    return score


# Criação do estudo e execução
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Alterar n_trials conforme necessário

# Exibição dos melhores parâmetros
print("Melhores parâmetros:", study.best_params)

[I 2024-11-25 02:59:21,517] A new study created in memory with name: no-name-d67c0734-ea9f-4212-8ca1-1742243a81c9
[W 2024-11-25 02:59:21,523] Trial 0 failed with parameters: {'C': 25.324802705132846, 'gamma': 0.9400000000000001, 'kernel': 'poly'} because of the following error: ValueError('A given column is not a column of the dataframe').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_indexing.py", line 361, in _get_column_indices
    col_idx = all_columns.get_loc(col)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/indexes/range.py", line 417, in get_loc
    raise KeyError(key)
KeyError: 'BMI'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-63-04ed8b87d276>", line 15, in objective
    pipeline.fit(x_train, y_tr

ValueError: A given column is not a column of the dataframe

In [ ]:
pipeline = Pipeline([
      #('outlier_remover', outlier_remover),
      ('preprocessor_svm', preprocessor_svm),
      ('svm', SVC())
  ])

C = 900
gamma = 0.11215038461268562
kernel=  'sigmoid'

pipeline.set_params(svm__C = C, svm__gamma = gamma, svm__kernel = kernel)
pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_temp)
score = f1_score(y_temp, y_pred)
print(score)
print(classification_report(y_temp, y_pred))


SyntaxError: incomplete input (<ipython-input-67-81cfc10a30d3>, line 1)

In [ ]:
print(score)

0.1935483870967742


In [ ]:
print(confusion_matrix(y_temp, y_pred))



[[124   8]
 [ 17   3]]
